**State Bank Deposits by Bank**

*Loading in Data*

The data for this plot comes from the quarterly bank financial data collected using the FDIC API. Then, in the Data Cleaning tab, we grouped the data by state, by bank, and by date meaning our data now showed the total assets, deposits, etc. that each bank in a given state held in a given quarter. This helped to shrink what was previously a massive dataset. However, that was still quite large. Therefore, that data file (./data/quarterly_banks_by_state.csv) is not in our github, however, you can see the code to create this file under ./code/data_cleaning/quarterly_data_cleaning.ipynb. Running this notebook will allow you to see exactly what data we are using.

In [75]:
# Import
import pandas as pd

# Reading in large dataframe
# See data cleaning
name_df = pd.read_csv("../data/quarterly_banks_by_state.csv", index_col=False)

# Labeling the top banks
top_banks = [
    "JPMORGAN CHASE BANK NA",
    "BANK OF AMERICA NA",
    "WELLS FARGO BANK NA",
    "CITIBANK NATIONAL ASSN",
]
name_df["top_bank"] = [
    bank if bank in top_banks else "Other" for bank in name_df["NAME"]
]

# Grouped Dataframe
df2 = name_df.groupby(["STNAME", "top_bank", "date", "decade"])[
    "DEPDOM", "ASSET"
].apply(lambda x: x.astype(int).sum())
df2 = df2.reset_index()

# Adding our theme colors for each bank
color_col = []

for bank in df2["top_bank"]:
    if bank == "JPMORGAN CHASE BANK NA":
        color = "#117ACA"
    elif bank == "BANK OF AMERICA NA":
        color = "#E61030"
    elif bank == "WELLS FARGO BANK NA":
        color = "#ffcc02"
    elif bank == "CITIBANK NATIONAL ASSN":
        color = "#003A72"
    else:
        color = "#118C4F"

    color_col.append(color)

df2["color"] = color_col
df2 = df2.reset_index()

# Displaying
df2.head()


/var/folders/hg/dd3yfd8j7vx8qtmvm42400j80000gn/T/ipykernel_22300/2112299546.py:20: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,index,STNAME,top_bank,date,decade,DEPDOM,ASSET,color
0,0,ALABAMA,Other,1992-03-31,1990,28888570,34804559,#118C4F
1,1,ALABAMA,Other,1992-06-30,1990,28782222,35151498,#118C4F
2,2,ALABAMA,Other,1992-09-30,1990,29034487,35810540,#118C4F
3,3,ALABAMA,Other,1992-12-31,1990,30059749,36943922,#118C4F
4,4,ALABAMA,Other,1993-03-31,1990,29888130,37040762,#118C4F


*Plot*

This plot went through a lot of variations. It began by included every state's total assets in five year windows with a slider to let the user move through the data. We then realize that five years was no where near enough time to see meaningful change so we expanded it to a decade long window. From there we realized that it would be better to just show all of the time at once, so the slider was scrapped. After doing some digging with the geo-spacial data, we found some interesting trends with the amount of money deposited in states you would not expect, such as South Dakota. In particular, we found that the biggest banks had tons of assets here. Quickly we realized this was worth exploring further so I split the data up by banks. However, it remained difficult to see how different the amounts of money was between states as you had to flip back and forth using the button. Finally, we switched to this subplot view which displays all of the states at once. The left column features the top three states by GDP while the right column features the three states we felt were most surprising in containing a ton of deposits despite being smaller states. They are all sorted vertically by GDP. 

Before the final submission, we will look to make some of the data selection more clearly with column titles. Additionally, we may look to add some annotations that point out some important dates. For example, Wells Fargo moving from California to South Dakota in 2003. We may also consider adding an additional line to each state displaying their full deposit totals to highlight the rapid drops of increases of gaining or losing a big bank.

In [78]:
# Imports
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Making a 3 by 2 Grid
fig = make_subplots(
    rows=3,
    cols=2,
    subplot_titles=(
        "California",
        "Ohio",
        "Texas",
        "North Carolina",
        "New York",
        "South Dakota",
    ),
    y_title="Total Deposits (USD)",
    x_title="Date",
)

# Making the bank names prettier
name_col = []

# Adding better names
for bank in df2["top_bank"]:
    if bank == "JPMORGAN CHASE BANK NA":
        name = "JP Morgan Chase"
    elif bank == "BANK OF AMERICA NA":
        name = "Bank of America"
    elif bank == "WELLS FARGO BANK NA":
        name = "Wells Fargo"
    elif bank == "CITIBANK NATIONAL ASSN":
        name = "Citibank"
    else:
        name = "All Other Banks"

    name_col.append(name)

df2["pretty_name"] = name_col

# Colors dictionary for easy access to our themed hex codes
# See theme.md for an explanation on these choicees
colors = dict(
    {
        "JP Morgan": "#117ACA",
        "Bank of America": "#E61030",
        "Wells Fargo": "#ffcc02",
        "Citibank": "#003A72",
        "All Other Banks": "#118C4F",
    }
)

# Adding the first subplot of California
# I figured out which bank to add here, and for all the others, with a simple unique banks functions as below
CA = df2[df2["STNAME"] == "CALIFORNIA"]
# print(CA.pretty_name.unique())

# Adding trace
# Only the first instance of each bank goes on the legend
fig.add_trace(
    go.Scatter(
        x=CA[CA["pretty_name"] == "All Other Banks"]["date"],
        y=CA[CA["pretty_name"] == "All Other Banks"]["DEPDOM"],
        name="All Other Banks",
        mode="lines",
        line=dict(color=colors["All Other Banks"]),
    ),
    row=1,
    col=1,
)

# Wells Fargo Trace
fig.add_trace(
    go.Scatter(
        x=CA[CA["pretty_name"] == "Wells Fargo"]["date"],
        y=CA[CA["pretty_name"] == "Wells Fargo"]["DEPDOM"],
        name="Wells Fargo",
        mode="lines",
        line=dict(color=colors["Wells Fargo"]),
    ),
    row=1,
    col=1,
)

# SOUTH DAKOTA
SD = df2[df2["STNAME"] == "SOUTH DAKOTA"]
# print(SD.pretty_name.unique())
fig.add_trace(
    go.Scatter(
        x=SD[SD["pretty_name"] == "Wells Fargo"]["date"],
        y=SD[SD["pretty_name"] == "Wells Fargo"]["DEPDOM"],
        name="Wells Fargo",
        mode="lines",
        line=dict(color=colors["Wells Fargo"]),
        showlegend=False,
    ),
    row=3,
    col=2,
)

fig.add_trace(
    go.Scatter(
        x=SD[SD["pretty_name"] == "Citibank"]["date"],
        y=SD[SD["pretty_name"] == "Citibank"]["DEPDOM"],
        name="Citibank",
        mode="lines",
        line=dict(color=colors["Citibank"]),
    ),
    row=3,
    col=2,
)

fig.add_trace(
    go.Scatter(
        x=SD[SD["pretty_name"] == "All Other Banks"]["date"],
        y=SD[SD["pretty_name"] == "All Other Banks"]["DEPDOM"],
        name="All Other Banks",
        mode="lines",
        line=dict(color=colors["All Other Banks"]),
        showlegend=False,
    ),
    row=3,
    col=2,
)

# TEXAS
TX = df2[df2["STNAME"] == "TEXAS"]
# Add traces
fig.add_trace(
    go.Scatter(
        x=TX[TX["pretty_name"] == "All Other Banks"]["date"],
        y=TX[TX["pretty_name"] == "All Other Banks"]["DEPDOM"],
        name="All Other Banks",
        mode="lines",
        line=dict(color=colors["All Other Banks"]),
        showlegend=False,
    ),
    row=2,
    col=1,
)


# NORTH CAROLINA
NC = df2[df2["STNAME"] == "NORTH CAROLINA"]

# Add traces
fig.add_trace(
    go.Scatter(
        x=NC[NC["pretty_name"] == "Bank of America"]["date"],
        y=NC[NC["pretty_name"] == "Bank of America"]["DEPDOM"],
        name="Bank of America",
        mode="lines",
        line=dict(color=colors["Bank of America"]),
    ),
    row=2,
    col=2,
)

fig.add_trace(
    go.Scatter(
        x=NC[NC["pretty_name"] == "All Other Banks"]["date"],
        y=NC[NC["pretty_name"] == "All Other Banks"]["DEPDOM"],
        name="All Other Banks",
        mode="lines",
        line=dict(color=colors["All Other Banks"]),
        showlegend=False,
    ),
    row=2,
    col=2,
)

# NEW YORK
NY = df2[df2["STNAME"] == "NEW YORK"]
# Add traces
fig.add_trace(
    go.Scatter(
        x=NY[NY["pretty_name"] == "Citibank"]["date"],
        y=NY[NY["pretty_name"] == "Citibank"]["DEPDOM"],
        name="Citibank",
        mode="lines",
        line=dict(color=colors["Citibank"]),
        showlegend=False,
    ),
    row=3,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=NY[NY["pretty_name"] == "All Other Banks"]["date"],
        y=NY[NY["pretty_name"] == "All Other Banks"]["DEPDOM"],
        name="All Other Banks",
        mode="lines",
        line=dict(color=colors["All Other Banks"]),
        showlegend=False,
    ),
    row=3,
    col=1,
)


# OHIO
OH = df2[df2["STNAME"] == "OHIO"]

# Add traces
fig.add_trace(
    go.Scatter(
        x=OH[OH["pretty_name"] == "JP Morgan Chase"]["date"],
        y=OH[OH["pretty_name"] == "JP Morgan Chase"]["DEPDOM"],
        name="JP Morgan Chase",
        mode="lines",
        line=dict(color=colors["JP Morgan"]),
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Scatter(
        x=OH[OH["pretty_name"] == "All Other Banks"]["date"],
        y=OH[OH["pretty_name"] == "All Other Banks"]["DEPDOM"],
        name="All Other Banks",
        mode="lines",
        line=dict(color=colors["All Other Banks"]),
        showlegend=False,
    ),
    row=1,
    col=2,
)

# Updating the layout for white backgrounds
fig.update_layout(template="plotly_white")

# Fixing y-ticks to be uniform
fig.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=[0, 500000000, 1000000000, 1500000000, 2000000000],
        ticktext=["0", "0.5B", "1B", "1.5B", "2B"],
    )
)

# adding one plot title and a legend title
fig.update_layout(
    title={
        "text": "Total Bank Deposits by State",
        "y": 0.9,
        "x": 0.45,
        "xanchor": "center",
        "yanchor": "top",
    },
    legend_title="Bank",
)


# Setting Custom bank names
# Help from https://stackoverflow.com/questions/71118467/how-to-include-trace-name-in-hovertemplate
for fig_scatter_data in fig.data:
    fig_scatter_data["customdata"] = [fig_scatter_data["name"]] * len(
        fig_scatter_data["x"]
    )

# Adding hovertemplate
fig.update_traces(
    hovertemplate="Bank: %{customdata} <br>Date: %{x} <br>Total Deposits: %{y:$,.0f}<extra></extra>"
)

# Display
fig.show()
fig.write_html("../img/plots/plot-3.html")


*Conclusions*

There is a lot to unpack with this plot. First, your eyes might be drawn to the rapid growth beginning in 2020 across the board but most obviously in Texas. Meanwhile, there is no obvious drop in deposits from the pandemic. Instead, we see the massive influx of cash into the banking system as a product of the pandemic and stimulus packages.

Secondly, and most importantly to our story here, is the differences in the size of the deposits in these states. You may even be confused at first and guess that the labels are on the wrong states because your intuition tells you that California, Texas, and New York should have significantly more cash in the bank than smaller states like South Dakota. However, that is where we were just as surprised as you are. As you may have noticed in our geo-spacial plot, huge banks are located in cities you would not expect. For example, Wells Fargo and Citibank are centered in Sioux Falls, South Dakota. As you can see, the deposits in these banks completely envelop the rest of the deposits in the state. Similarly, in Ohio and North Carolina, the large banks located there make up the majority of the total deposits in the state. 

Why these banks are located in these unlikely locations is up to speculations, but regulation could certainly play a role, namely, corporate tax rates. South Dakota and Ohio both lack any corporate taxes and North Carolina has one of the lowest in the country (https://taxfoundation.org/publications/state-corporate-income-tax-rates-and-brackets/). These corporate taxes may help explain why Wells Fargo moved from California to South Dakota in 2003, or why Citibank followed them to Sioux Falls in 2011. There certainly could be more at play, however, tax breaks and heavy regulation may help explain why big banks are leaving the country's richest states for ones that will not tax them. 

*References*

- Plots made with plotly: https://plotly.com/python/

- Hovertip Help: https://stackoverflow.com/questions/71118467/how-to-include-trace-name-in-hovertemplate

- Data cleaned with pandas: https://pandas.pydata.org/

- Corporate Tax Data: https://taxfoundation.org/publications/state-corporate-income-tax-rates-and-brackets/